In [1]:
import pandas as pd
import numpy as np

In [2]:
cols = [
"geonameid",
"name",
"asciiname",
"alternatenames",
"lat",  
"lon",
"feature class",
"feature code",   
"country code",    
"cc2",    
"admin1 code",
"admin2 code",     
"admin3 code",     
"admin4 code",     
"population",     
"elevation",      
"dem",       
"timezone",
"modification date",
]

In [3]:
all_countries = pd.read_csv('data/allCountries.zip', sep='\t', header=None, names=cols)

/Users/hhoferic/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (9,10,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
country_codes = pd.read_csv('data/countrycodes.txt', sep='\t')

In [5]:
admin_codes = pd.read_csv('data/admin1.txt', sep='\t', index_col=None)

In [6]:
name_codes = pd.read_csv('data/geonames_codes.txt', sep='\t')

In [7]:
admin_codes["country"] = admin_codes.code.apply(lambda x: x.split(".")[0].strip())

In [8]:
admin_codes["province"] = admin_codes.code.apply(lambda x: x.split(".")[1].strip())

In [9]:
name_codes['key'] = name_codes.code.apply(lambda x: x.split(".")[1])

In [10]:
merged = all_countries.merge(country_codes[['ISO', 'Country']], left_on='country code', right_on='ISO', how='left')

In [11]:
merged = merged.merge(name_codes[['key', 'item']], left_on='feature code', right_on='key', how='left')

In [12]:
merged = merged.merge(admin_codes[['country', 'province', 'name']], left_on=['ISO','admin1 code'], right_on=['country', 'province'], how='left')

In [13]:
def row_to_text(row):
#     print(row)
    row = row.to_dict()
    if not pd.isna(row['province']) and not row['population'] == 0:
        return_string = f"{row['name_x']} is a {row['item']} in {row['name_y']} in {row['Country']} with a population of {row['population']}."
    elif pd.isna(row['province']) and not row['population'] == 0:
        return_string = f"{row['name_x']} is a {row['item']} in {row['Country']} with a population of {row['population']}."
    elif not pd.isna(row['province']) and row['population'] == 0:
        return_string = f"{row['name_x']} is a {row['item']} in {row['name_y']} in {row['Country']}."
    elif pd.isna(row['province']) and row['population'] == 0:
        return_string = f"{row['name_x']} is a {row['item']} in {row['Country']}."
    else:
        return_string = f"{row['name_x']} is a {row['item']} in {row['Country']}."
    if not pd.isna(row['timezone']):
        split = row['timezone'].split("/")
        zone = split[0]
        major_city = split[1] if len(split)==2 else " ".join(split[1:])
        major_city = major_city.replace("_", " ")
        return_string += f" It's located in {zone} and the nearest major city is {major_city}."
    return return_string
    

In [14]:
texts = merged.apply(row_to_text, axis=1)
# texts = merged.name_x + " or " + merged.alternatenames + " is a " + merged.item + " in " + merged.Country

In [15]:
texts = texts.rename('text').to_frame()

In [16]:
texts['lat'] = merged.lat
texts['lon'] = merged.lon

In [17]:
texts = texts[~texts.text.isna()]

In [18]:
texts.to_csv('data/geonames_data.csv')